# Object Detection Model Training

Hello, user! This notebook will walk you through the training process of an object detection model using transfer learning from Tensorflow Object Detection API. Please run all the relevant cells in order. Text paragraphs like these will not fully substitute conventional Python comments but will serve as additonal commentary on the process. <br>
Ensure that the kernel is Python version 3.9.6

### Step 1: Import relevant libraries

Not much to be said here, just importing libraries. Justification for libraries used are found in the comments.

In [ ]:
# Import libraries
import os # To run terminal commands
import tensorflow as tf # Google's Machine Learning frameowrk. using for Transfer training.
from object_detection.utils import config_util # Add-on for Tensorflow
from object_detection.protos import pipeline_pb2 # Add-on for Tensorflow
from google.protobuf import text_format # Add-on for Tensorflow

### Step 2: Generate TF records

TF record files hold the necessary data for training a model. <br>
Ensure that your images and labels for each individual image are in the following folders before proceeding with this step. <br>
Training images: `./Tensorflow/workspace/images/train`<br>
Test images: `./Tensorflow/workspace/images/test`

In [ ]:
print("Generating train.record")
os.system("python Tensorflow/scripts/generate_tfrecord.py -x Tensorflow/workspace/images/train -l Tensorflow/Workspace/annotations/label_map.pbtxt -o Tensorflow/workspace/annotations/train.record")
print("Generating test.record")
os.system("python Tensorflow/scripts/generate_tfrecord.py -x Tensorflow/workspace/images/test -l Tensorflow/Workspace/annotations/label_map.pbtxt -o Tensorflow/workspace/annotations/test.record")

### Step 3: Create the label map

The label map is the file that contains the labels for the different category of items. The file needs to be generated according to the labels to be used.

In [ ]:
# <------------------------------------Generate label map------------------------------------>
while True: # running input validation for an integer
  num = ("Enter the number of types of items/objects (between 0 to 100) to be detected: ") # Get user input on the number of labels
  try:
    num = int(num)
    if num > 0 and num < 100:
      break
    else:
      print("Error: input format is invalid. Please try again.")
  except:
    print("Error: input format is invalid. Please try again.")
label_map = []
for x in num:
  label = input("Enter label name for item " + str(x) + ": ") # Get user input on label names
  label_map.append({"name": label, "num": x})

In [ ]:
with open("Tensorflow/workspace/annotations" + '/label_map.pbtxt', 'w') as f: # Write label map file
    for x in label_map:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(x['name']))
        f.write('\tid:{}\n'.format(x['id']))
        f.write('}\n')

### Step 4: Create and set-up relevant paths for model directory

Input the name for the model and the code will create the model directpry, as well as import the template model

In [ ]:
# <------------------------------------Create model directory------------------------------------>
model_name = input("Enter name for the model") # Get use input for custom model name
os.system("mkdir Tensorflow/workspace/models/" + model_name) # Creates directory to store all model-related files
os.system("cp Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config Tensorflow/workspace/models/" + model_name) # Copy template model config file into custom model

### Step 5: Set up and configure config file

If all has been done successfully, just run the following block of code.

In [ ]:
# <------------------------------------Configure config file------------------------------------>
config_file_path = "Tensorflow/workspace/models/" + model_name + "/pipeline.config"
config = config_util.get_configs_from_pipeline_file(config_file_path)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_file_path, "r") as f:                                                      
    proto_str = f.read()                                                                                                  
    text_format.Merge(proto_str, pipeline_config)

# <------------------------------------Settings for config file------------------------------------>
while True:
    batch_size = input("Default batch size is 4. Max batch size is 10. Enter custom batch size: ") # User input for batch size for training
    if str(batch_size) == "":
        batch_size = 4
        break
    try:
        batch_size = int(batch_size)
        if batch_size > 0 and batch_size < 11:
            break
        else:
            print("Error: Input format is invalid. Please try again.")
    except:
        print("Error: Input format is invalid. Please try again.")


pipeline_config.model.ssd.num_classes = num
pipeline_config.train_config.batch_size = batch_size
pipeline_config.train_config.fine_tune_checkpoint = "Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= "Tensorflow/Workspace/annotations/label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ["Tensorflow/Workspace/annotations/train.record"]
pipeline_config.eval_input_reader[0].label_map_path = "Tensorflow/Workspace/annotations/label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ["Tensorflow/Workspace/annotations/test.record"]

# Write to the config file
config_text = text_format.MessageToString(pipeline_config)                                                                       
with tf.io.gfile.GFile(config_file_path, "wb") as f:
    f.write(config_text)

### Step 6: Generating Terminal command for training

You're done! Run the following block of code to generate the Terminal command. Paste the following command into the Terminal from this directory and training will commence.

In [ ]:
print("Preparation for training complete. Training shall now commence")
print("Please copy and paste the following program into your Terminal within this directory.")
print("""python Tensorflow/workspace/model_main_tf2.py --model_dir=Tensorflow/workspace/models/{} --pipeline_config_path=Tensorflow/workspace/models/{}/pipeline.config --num_train_steps=10000""".format(model_name,model_name)) # Generate Terminal command for user to run
print("This program will now terminate. All the best with the training!")
exit()